# Working With Scientific Notation

The `exponent_format` parameter in the `theme(...)` function can be used to configure the way "exponent notation" looks like on plot.

Available values:

- `"e"` for "e" notation (e.g. 1e+6);
- `"pow_full"` for "power" notation (e.g. $1 \cdot 10^6$). This will enable superscript formatting for the exponent;
- `"pow"` works as `"pow_full"` but will shorten powers of 10 (e.g. $10^6$ instead of $1 \cdot 10^6$).

The "exponent format" is automatically applied to each value formatted in scientific notation, regardless whether the format is user-defined or chosen automatically based on the data. This format affects every part of a plot, including geoms, scales, labels, and tooltips.

In [1]:
%useLatestDescriptors
%use dataframe
%use lets-plot

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.9.0-SNAPSHOT. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.5.1.

In [3]:
import org.jetbrains.letsPlot.intern.Plot
import org.jetbrains.letsPlot.intern.Scale

In [4]:
val amuCoeffNG = 1.6605402 * 10.0.pow(-15)

In [5]:
val df = DataFrame.readCSV("https://github.com/JetBrains/lets-plot-docs/" +
                           "raw/refs/heads/master/data/chemical_elements.csv").let { df ->
    df["Element", "Symbol", "Type", "Atomic Weight", "Atomic Radius"]
        .rename("Element", "Atomic Weight", "Atomic Radius")
        .into("Name", "Weight (ng)", "Radius")
        .update { "Weight (ng)"<Double>() }.with { amuCoeffNG * it }
}
val dataMap = df.toMap()
df.head()

Name,Symbol,Type,Weight (ng),Radius
Hydrogen,H,Nonmetal,0.000000,0.790000
Helium,He,Noble Gas,0.000000,0.490000
Lithium,Li,Alkali Metal,0.000000,2.100000
Beryllium,Be,Alkaline Earth Metal,0.000000,1.400000
Boron,B,Metalloid,0.000000,1.200000


In [6]:
val dMin = -7
val dMax = 6
val values = (dMin..dMax).map { d -> 10.0.pow(d) }

fun hourglassPlot(title: String, numberFormat: String? = null): Plot {
    return letsPlot() { y = values; label = values } +
        geomText(labelFormat = numberFormat, size = 10) +
        scaleYContinuous(limits = Pair(10.0.pow(dMin - 1), 10.0.pow(dMax + 1)),
                         trans = "log10", format = numberFormat) +
        ggtitle(title) +
        themeClassic()
}

In [7]:
val chemicalElementsPlot = letsPlot(dataMap) +
    geomLabel(alpha = .75, fontface = "bold") {
        x = "Weight (ng)"; y = "Radius"; label = "Symbol"; fill = "Type"
    } +
    scaleFillBrewer(guide = guideLegend(ncol = 2)) +
    ggsize(1000, 500) +
    theme(axisTextX = elementText(size = 16, face = "bold"),
          legendKeySize = 12, legendText = elementText(size = 9))
        .legendPosition(.91, .82)

fun formattedXScale(numberFormat: String? = null): Scale {
    return scaleXContinuous(limits = Pair(0.0, 4 * 10.0.pow(-13)),
                            breaks = (0..8).map { (it / 2.0) * 10.0.pow(-13) },
                            format = numberFormat)
}

## Default Plot

In [8]:
chemicalElementsPlot + formattedXScale()

## Specify Format

In [9]:
gggrid(listOf(
    hourglassPlot("Default format"),
    hourglassPlot("format='g'", "g")
))

In [10]:
chemicalElementsPlot + formattedXScale("g")

## Switch Exponent Format to Power Degree

In [11]:
gggrid(listOf(
    hourglassPlot("Default exponentFormat", "g"),
    hourglassPlot("exponentFormat='pow'", "g") + theme(exponentFormat = "pow"),
    hourglassPlot("exponentFormat='pow_full'", "g") + theme(exponentFormat = "pow_full")
))

In [12]:
chemicalElementsPlot + formattedXScale("g") + theme(exponentFormat = "pow")

## Specify the Limits From Which the Scientific Notation Used

In [13]:
gggrid(listOf(
    hourglassPlot("Default limits", "g") + theme(exponentFormat = "pow"),
    hourglassPlot("""Scientific notation for \( x \leq 10^{-3} \) and \( x \geq 10^3 \)""", "g") +
        theme(exponentFormat = listOf("pow", -3, 3))
))

In [14]:
chemicalElementsPlot + formattedXScale("g") + theme(exponentFormat = listOf("pow", -14, null))